# Column Transformer


In machine learning, a column transformer is a preprocessing step that allows different transformations to be applied to different columns (or subsets of columns) of a dataset. It is particularly useful when dealing with datasets that have heterogeneous data types or require different preprocessing steps for different features.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']),
                                                    df['has_covid'],
                                                    test_size=0.2,
                                                    random_state=0)

x_train.head()

,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
3,31,Female,98.0,Mild,Kolkata
71,75,Female,104.0,Strong,Delhi
45,72,Male,99.0,Mild,Bangalore


### 📌If we carry out analysis without Column transformer

In [4]:
from sklearn.impute import SimpleImputer
si=SimpleImputer()

x_train_fever = si.fit_transform(x_train[['fever']])
x_test_fever = si.fit_transform(x_train[['fever']])

x_train_fever.shape

(80, 1)

In [5]:
from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder(categories=[['Mild', 'Strong']])

x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.fit_transform(x_test[['cough']])

x_train_cough.shape

(80, 1)

In [6]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='first',sparse=False)
x_train_gender_city = ohe.fit_transform(x_train[['gender', 'city']])
x_test_gender_city = ohe.fit_transform(x_test[['gender', 'city']])

x_train_gender_city.shape

/home/codespace/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [7]:
x_train_age = x_train.drop(columns=['gender', 'city', 'fever', 'cough']).values

x_test_age = x_test.drop(columns=['gender','city','fever','cough']).values

x_train_age.shape

(80, 1)

In [8]:
x_train_transformed = np.concatenate((x_train_age, x_train_fever, x_train_gender_city, x_train_cough), axis=1)
# x_test_transformed = np.concatenate((x_test_age, x_test_fever, x_test_gender_city, x_test_cough), axis=1)

x_train_transformed.shape

(80, 7)

### 📌 If we carry out analysis with Column transformer

In [9]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse=False, drop='first'), ['gender', 'city'])
], remainder='passthrough')

transformer.fit_transform(x_train).shape

/home/codespace/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)